# SVM  

This will start with using SKLearn, but might change to Tensorflow

For now, only one stock will be used, so for now my file that takes in data will find that one stock, and it will be one from the sp500

In [11]:
import pandas as pd
import os
import glob
from pathlib import Path

curr_data_folder = Path('stock_market_data/sp500/csv/')

symbol_csvs_paths = list(curr_data_folder.glob('*.csv'))

In [12]:

import numpy as np

def stock_csv_parser(fileName):

    symbol_df = pd.read_csv(fileName)

    # I want to get the percentage change, so if I wanted to make a threshold such as 2%
    # I will first drop any open if it is zero just in case to prevent division by zero
    symbol_df = symbol_df[symbol_df['Open'] != 0]
    y = symbol_df['Close'] / symbol_df['Open']



    y = [1 if each > 1.025 else 0 for each in y]

    # print(y)
    # print("data points:", str(len(y)), "\t Results as success:", np.count_nonzero(y))
    
    return symbol_df, y



Now that we have one of the stock in, we will try to moddify the data, then train it with svm for now
I will come back later to get it to do more than just one stock, this is for quicker testing for now

In [13]:
from sklearn.preprocessing import StandardScaler

def scaler_function(symbol_df):
    try:
        symbol_df = symbol_df.drop(['Date'], axis = 1)
        symbol_df = symbol_df.dropna()
    except:
        print("Column missing")
    
    try:
        symbol_df = symbol_df.drop(['Adjusted Close'], axis = 1)
    except:
        print("Missing Column")

    # print(symbol_df.iloc[0])

    # print(symbol_df)

    scaler = StandardScaler()
    X = scaler.fit_transform(symbol_df)

    # print(X)
    return X


I am unsure if I should do PCA first or append 10 days to each other then do it
I suspect I will do PCA after since each day will be highly correlated to the previous day

In [14]:

def X_day_appender(day_info, day_ct=10):
    result = []
    for i in range(len(day_info) - (day_ct)):
        result.append(day_info[i:i+day_ct, :].flatten())
    return result

I want to now use PCA because this data will all naturally be correlated

In [15]:
from sklearn.decomposition import PCA

def principal_component_analyzer(X, n=7):
    pca = PCA(n_components=n)
    X_PCA = pca.fit_transform(X)
    return X_PCA

# print(X_PCA[0], len(X_PCA), len(y))


Now that I have done PCA, I want to split my data so that future data is not trained on

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

def data_spliter(X_PCA, y, day_ct):
    # This will split up my data, while keeping it in order
    X_train, X_test, y_train, y_test = train_test_split(X_PCA, y, test_size=0.2, shuffle=False)

    # I am removing some that are close or are in the train set at least part of the way
    X_test = X_test[day_ct+5:]
    y_test = y_test[day_ct+5:]

    # I am then shuffleing the data, thought that might have only been NN that needed this step, I am unsure
    X_train, y_train = shuffle(X_train, y_train, random_state=42)
    X_test, y_test = shuffle(X_test, y_test, random_state=42)
    return X_train, X_test, y_train, y_test



Now that I have done quite a bit to the data, I will train it using SVM
I want to see if the results can have a high percision
I think percision is what I want the most because I do not want to have bad days be prediceted as good days

I will first be trying svc, because I want a classification
I might move to one class svm because there is a great deal of the results not meeting the threshold

I am using svc and not linear svc, because the documentation suggested linear svc if you have more than 10s of thousands of samples
Which when only doing one stock with this data set, there is not too many samples

In [17]:
from sklearn.svm import SVC

def SVM_fitting(X_train, y_train, X_test):
    clf = SVC(gamma='auto')
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    return y_pred


Now that it is train, I want to see what I can get out of the results

In [18]:
from sklearn.metrics import precision_score, recall_score, accuracy_score

def score_returner(y_test, y_pred):
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

    # print("Accuracy: ", str(accuracy),  "\tPrecision: ", str(precision), "\tRecall: ", str(recall), "\tNumber of predicted days: ", str(np.count_nonzero(y_pred))) 
    return accuracy, recall, precision

So far the results are not great, I will continute to work to see if I can get any sort of result from this
I have tried a few stocks and General Electric is returning the best results so far

I plan to also add in a way to see the change in value if when predicted to be a good day, the stock is bought at open and sold at close
I plan to also try to test all sp500 stocks at once to see which ones give the best results

In [19]:
def symbol_train_test(file_path, day_ct=10):
    symbol_df, y = stock_csv_parser(file_path)
    X = scaler_function(symbol_df)
    X = principal_component_analyzer(X,3)
    X = X_day_appender(X, day_ct=day_ct)
    y = y[day_ct:]
    X = principal_component_analyzer(X,15)
    X_train, X_test, y_train, y_test = data_spliter(X, y, day_ct)
    y_pred = SVM_fitting(X_train, y_train, X_test)
    accuracy, recall, precision = score_returner(y_test, y_pred)  
    return accuracy, recall, precision


In [20]:
symbol_csvs_paths

# try:
file = open("SVM_Scores.csv", "a")
curr_string_to_write = "symbol,accuracy,recall,precision\n"
file.write(curr_string_to_write)
file.close()

for each in symbol_csvs_paths:
    symbol = os.path.splitext(os.path.basename(each))
    # try:
    print("\nTraining on: ", symbol[0])
    accuracy, recall, precision = symbol_train_test(each, day_ct=15)
    curr_string_to_write = f"{symbol[0]},{accuracy},{recall},{precision}\n"
    file = open("SVM_Scores.csv", "a")
    file.write(curr_string_to_write)
    file.close()

# file.close()
# except:
#     print("Error")


Training on:  RHI


/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Training on:  LDOS

Training on:  GWW


/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Training on:  HII

Training on:  PM


...

/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Training on:  VTR


/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Training on:  ALK


KeyboardInterrupt: 